In [ ]:
import pandas as pd
import numpy as np
import os
import typing
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error

from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.losses import MeanSquaredError
from keras.optimizers import Adam

In [ ]:
DATA_PATH = 'data_cars/'
all_files = os.listdir(DATA_PATH)

all_dataframes = []
for index, file in enumerate(all_files):
    print(f"Reading file: {file}")
    file_name = file.split('.')[0]
    df = pd.read_csv(DATA_PATH + file, sep=';')

    df['date'] = pd.to_datetime(df[file_name], format='%Y-%m-%d %H:%M')
    df = df.drop(columns=[file_name])

    df = df.set_index('date')
    df.columns = [f"{file_name}_{col}" for col in df.columns if col != 'date']
    all_dataframes.append(df)
    print(f"Finished reading file: {file}, shape = {df.shape}")

combined_df = pd.concat(all_dataframes, axis=1)
combined_df.fillna(method='ffill', inplace=True)
combined_df['hour'] = combined_df.index.hour
combined_df['day_of_week'] = combined_df.index.dayofweek

data = np.array(combined_df, dtype=float)[:, :-2]

singleScaler = StandardScaler()
singleSensorData = singleScaler.fit_transform(data[:, 0].reshape(-1,1))
singleIntScaler = StandardScaler()
singleIntData = singleIntScaler.fit_transform(data[:, :7])
scaler = StandardScaler()
data = scaler.fit_transform(data)

In [ ]:
timestamps = combined_df.index
dataset = combined_df.to_numpy()

In [ ]:
def splitSequence(seq, n_steps):

    #Declare X and y as empty list
    X = []
    y = []

    for i in range(len(seq)):
        #get the last index
        lastIndex = i + n_steps

        #if lastIndex is greater than length of sequence then break
        if lastIndex > len(seq) - 1:
            break

        # Create input and output sequence
        # Last 2 columns are time of day and day of week
        seq_X, seq_y = seq[i:lastIndex], seq[lastIndex]

        #append seq_X, seq_y in X and y list
        X.append(seq_X)
        y.append(seq_y)
        #Convert X and y into numpy array
    X = np.array(X)
    y = np.array(y)

    return X,y

In [ ]:
num_of_steps = data.shape[0]
train_size = 0.6
val_size = 0.15
shuffle = False
look_back = 80

x, y = splitSequence(data, look_back)
x_one, y_one = splitSequence(singleSensorData, look_back)
x_int, y_int = splitSequence(singleIntData, look_back)

if shuffle:
    idx = np.random.permutation(len(x))
    x,y = x[idx], y[idx]

num_train = int(num_of_steps * train_size)
num_val = int(num_of_steps * val_size)

x_train, y_train = x[:num_train], y[:num_train]
x_val, y_val = x[num_train:num_train + num_val], y[num_train:num_train + num_val]
x_test, y_test = x[num_train + num_val:], y[num_train + num_val:]

x_train_single, y_train_single = x_one[:num_train], y_one[:num_train]
x_val_single, y_val_single = x_one[num_train:num_train + num_val], y_one[num_train:num_train + num_val]
x_test_single, y_test_single = x_one[num_train + num_val:], y_one[num_train + num_val:]

x_train_singleInt, y_train_singleInt = x_int[:num_train], y_int[:num_train]
x_val_singleInt, y_val_singleInt = x_int[num_train:num_train + num_val], y_int[num_train:num_train + num_val]
x_test_singleInt, y_test_singleInt = x_int[num_train + num_val:], y_int[num_train + num_val:]

print(x_train.shape)
print(x_val.shape)
print(x_test.shape)

In [ ]:
#Add params to do optimizing at the top
input_dim = 1
output_size = 1
units = 60

input = keras.Input((look_back, input_dim))
#return sequences is necessary for sequential LSTM layers
lstm1 = LSTM(units, return_sequences=True)(input)
lstm2 = LSTM(units)(lstm1)
out = Dense(output_size)(lstm2)
model = keras.models.Model(inputs=input, outputs=out)
model.summary()

model.compile(
    loss=MeanSquaredError(),
    optimizer=Adam(learning_rate=0.001),
    metrics=[keras.metrics.RootMeanSquaredError()],
)

history = model.fit(
    x=x_train_single,
    y=y_train_single,
    validation_data=(x_val_single, y_val_single),
    epochs=500,
    #makes the training stop early if it notices no improvements on the validation set 10 times in a row, to prevent overfitting
    callbacks=[keras.callbacks.EarlyStopping(patience=10)],
)

# make predictions
trainPredict = model.predict(x_train_single)
testPredict = model.predict(x_test_single)
# invert predictions
trainPredict = singleScaler.inverse_transform(trainPredict)
trainY = singleScaler.inverse_transform(y_train_single)
testPredict = singleScaler.inverse_transform(testPredict)
testY = singleScaler.inverse_transform(y_test_single)
# calculate root mean squared error
trainScore = np.sqrt(mean_squared_error(trainY, trainPredict))
print(f'Train Score: {trainScore:.2f} RMSE')
testScore = np.sqrt(mean_squared_error(testY, testPredict))
print(f'Test Score: {testScore:.2f} RMSE')

In [ ]:
plt.plot(testY, label='actual')
plt.plot(testPredict, label='predict')
plt.legend()

In [ ]:
#Add params to do optimizing at the top
input_dim = x_train_singleInt.shape[2]
output_size = 1
units = 60

input = keras.Input((look_back, input_dim))
#return sequences is necessary for sequential LSTM layers
lstm1 = LSTM(units, return_sequences=True)(input)
lstm2 = LSTM(units)(lstm1)
out = Dense(output_size)(lstm2)
model = keras.models.Model(inputs=input, outputs=out)
model.summary()

model.compile(
    loss=MeanSquaredError(),
    optimizer=Adam(learning_rate=0.001),
    metrics=[keras.metrics.RootMeanSquaredError()],
)

history = model.fit(
    x=x_train_singleInt,
    y=y_train_single,
    validation_data=(x_val_singleInt, y_val_single),
    epochs=500,
    #makes the training stop early if it notices no improvements on the validation set 10 times in a row, to prevent overfitting
    callbacks=[keras.callbacks.EarlyStopping(patience=5)],
)

# make predictions
trainPredict = model.predict(x_train_singleInt)
testPredict = model.predict(x_test_singleInt)
# invert predictions
trainPredict = singleScaler.inverse_transform(trainPredict)
trainY = singleScaler.inverse_transform(y_train_single)
testPredict = singleScaler.inverse_transform(testPredict)
testY = singleScaler.inverse_transform(y_test_single)
# calculate root mean squared error
trainScore = np.sqrt(mean_squared_error(trainY, trainPredict))
print(f'Train Score: {trainScore:.2f} RMSE')
testScore = np.sqrt(mean_squared_error(testY, testPredict))
print(f'Test Score: {testScore:.2f} RMSE')

In [ ]:
#Add params to do optimizing at the top
input_dim = data.shape[1]
output_size = y_train.shape[1]
units = 400

input = keras.Input((look_back, input_dim))
#return sequences is necessary for sequential LSTM layers
lstm1 = LSTM(units, return_sequences=True)(input)
lstm2 = LSTM(units)(lstm1)
out = Dense(output_size)(lstm2)
model = keras.models.Model(inputs=input, outputs=out)
model.summary()

model.compile(
    loss=MeanSquaredError(),
    optimizer=Adam(learning_rate=0.001),
    metrics=[keras.metrics.RootMeanSquaredError()],
)

history = model.fit(
    x=x_train,
    y=y_train,
    validation_data=(x_val, y_val),
    epochs=500,
    #makes the training stop early if it notices no improvements on the validation set 10 times in a row, to prevent overfitting
    callbacks=[keras.callbacks.EarlyStopping(patience=10)],
)

# make predictions
trainPredict = model.predict(x_train)
testPredict = model.predict(x_test)
# invert predictions
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform(y_train)
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform(y_test)
# calculate root mean squared error
trainScore = np.sqrt(mean_squared_error(trainY, trainPredict))
print(f'Train Score: {trainScore:.2f} RMSE')
testScore = np.sqrt(mean_squared_error(testY, testPredict))
print(f'Test Score: {testScore:.2f} RMSE')